In [9]:
from keras.models import Input, Model
from keras.layers import Conv2D, Concatenate, MaxPooling2D, Conv2DTranspose
from keras.layers import UpSampling2D, Dropout, BatchNormalization

'''
U-Net:
img_shape: (height, width, channels)
out_ch: number of output channels
start_ch: number of channels of the first conv
depth: zero indexed depth of the U-structure
inc_rate: rate at which the conv channels will increase
activation: activation function after convolutions
dropout: amount of dropout in the contracting part
batchnorm: adds Batch Normalization if true
maxpool: use strided conv instead of maxpooling if false
upconv: use transposed conv instead of upsamping + conv if false
residual: add residual connections around each conv block if true
'''

def conv_block(m, dim, acti, bn, res, do=0):
    n = Conv2D(dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(do)(n) if do else n
    n = Conv2D(dim, 3, activation=acti, padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    return Concatenate()([m, n]) if res else n

def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
    if depth > 0:
        n = conv_block(m, dim, acti, bn, res)
        m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
        m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
        if up:
            m = UpSampling2D()(m)
            m = Conv2D(dim, 2, activation=acti, padding='same')(m)
        else:
            m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
        n = Concatenate()([n, m])
        m = conv_block(n, dim, acti, bn, res)
    else:
        m = conv_block(m, dim, acti, bn, res, do)
    return m

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu',
         dropout=0.5, batchnorm=False, maxpool=True, upconv=False, residual=False):
    i = Input(shape=img_shape)
    o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
    o = Conv2D(out_ch, 1, activation='sigmoid')(o)
    return Model(inputs=i, outputs=o)

In [10]:
model = UNet((256,256,3))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 256, 256, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_34[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_35[0][0]                  
__________________________________________________________________________________________________
conv2d_36 

In [11]:
import os
import cv2 
from pathlib import Path
import numpy as np 
import matplotlib.pyplot as plt

In [12]:
SIZE = (256, 256)
images = []
masks = []

In [13]:
def fix_mask(mask):
    mask[mask < 100] = 0.0
    mask[mask >= 100] = 255.0

def draw_image_gray(img):
    plt.imshow(img,cmap='gray')
    plt.show()
    
def draw_image_rgb(img):
    plt.imshow(img)
    plt.show()
    
def preprocess_rgb(img):
    img = img[:,:,:3]
    img = cv2.resize(img, SIZE)
    img = img/255.
    images.append(img)
    
def preprocess_gray(mask):
    mask = mask[:,:,:3]
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    fix_mask(mask)
    mask = cv2.resize(mask, SIZE)
    mask = mask/255.
    mask = np.expand_dims(mask, axis=2)
    masks.append(mask)

In [14]:
p = Path("Desktop/SRIB/img")

cnt = 0
for i in sorted(p.glob("*.jpg")):
    string = str(i)
    img = cv2.imread(string)
    np_img = np.array(img)
    preprocess_rgb(np_img)
    cnt += 1

cnt1 = 0
for i in sorted(p.glob("*.png")):
    string = str(i)
    np_mask = cv2.imread(string)
    preprocess_gray(np_mask)
    cnt1 += 1

p = Path("Desktop/SRIB/ShadowImages")
for i in sorted(p.glob("*.jpg")):
    string = str(i)
    img = cv2.imread(string)
    np_img = np.array(img)
    preprocess_rgb(np_img)
    cnt += 1

p = Path("Desktop/SRIB/ShadowMasks")
for i in sorted(p.glob("*.png")):
    string = str(i)
    np_mask = cv2.imread(string)
    preprocess_gray(np_mask)
    cnt1 += 1

X = np.array(images)
Y = np.array(masks)

In [15]:
print(X.shape)
print(Y.shape)

(4440, 256, 256, 3)
(4440, 256, 256, 1)


In [21]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
earlystopper = EarlyStopping(monitor='val_loss',
                             patience=8,
                             verbose=1,
                             min_delta=1e-4)

reducelronplateau = ReduceLROnPlateau(monitor='val_loss',
                                      factor=0.1,
                                      patience=4,
                                      verbose=1,
                                      epsilon=1e-4)

checkpointer = ModelCheckpoint(monitor='val_loss',
                               filepath='best_weights_new1.hdf5',
                               verbose=1,
                               save_best_only=True,
                               save_weights_only=True)

C:\Anaconda\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [ ]:
from keras.optimizers import *
model.compile(Adam(lr = 1e-4), loss = 'binary_crossentropy')
model.fit(X, Y, validation_split=0.1, batch_size=32, epochs=50, callbacks=[earlystopper, checkpointer, reducelronplateau])

Train on 3996 samples, validate on 444 samples
Epoch 1/50
3996/3996 [==============================] - 547s 137ms/step - loss: 0.3400 - val_loss: 0.2424

Epoch 00001: val_loss improved from inf to 0.24241, saving model to best_weights_new1.hdf5
Epoch 2/50
3996/3996 [==============================] - 497s 124ms/step - loss: 0.2259 - val_loss: 0.2271

Epoch 00002: val_loss improved from 0.24241 to 0.22714, saving model to best_weights_new1.hdf5
Epoch 3/50
3996/3996 [==============================] - 497s 124ms/step - loss: 0.2047 - val_loss: 0.1875

Epoch 00003: val_loss improved from 0.22714 to 0.18745, saving model to best_weights_new1.hdf5
Epoch 4/50
3996/3996 [==============================] - 498s 125ms/step - loss: 0.1894 - val_loss: 0.1743

Epoch 00004: val_loss improved from 0.18745 to 0.17434, saving model to best_weights_new1.hdf5
Epoch 5/50
3996/3996 [==============================] - 497s 124ms/step - loss: 0.1867 - val_loss: 0.1766

Epoch 00005: val_loss did not improve from